In [ ]:
# Dec. 13, 2018 -- inksci@qq.com

# conda.env: tf-gpu-v4 (inksci-laptop-nvidia)
# Python 2

In [35]:
tf.__version__

'1.4.1'

In [1]:
import numpy as np
import random
import tensorflow as tf

In [2]:
import os
import csv
import itertools

In [3]:
import tensorflow.contrib.slim as slim

In [4]:
# Networks design, 神经网络设计

In [5]:
x =  tf.placeholder(shape=[None, 400, 400, 3],dtype=tf.float32)

In [6]:
conv1 = slim.convolution2d( \
  inputs=x,num_outputs=20,\
  kernel_size=[8,8],stride=[1,1],padding='VALID', \
  biases_initializer=None,scope='conv1')

In [7]:
print( conv1.shape )

(?, 393, 393, 20)


In [8]:
spatial_softmax = tf.contrib.layers.spatial_softmax( conv1 )  

In [9]:
print( "spatial_softmax.shape: ", spatial_softmax.shape )

('spatial_softmax.shape: ', TensorShape([Dimension(None), Dimension(40)]))


In [10]:
f1 = tf.layers.dense(spatial_softmax, 30, activation = tf.nn.relu, name = 'f1')

In [11]:
f2 = tf.layers.dense(f1, 2, name = 'f2')

In [12]:
y_input = tf.placeholder("float",[None,2])

In [13]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

In [14]:
loss = tf.reduce_mean(tf.square(y_input - f2))

In [15]:
train = optimizer.minimize( loss )

In [16]:
# load image from file

In [17]:
import cv2
import numpy as np

In [18]:
im = cv2.imread( "./data-set/data-moveImg-400x400/moveImg-0.bmp" )

In [19]:
im.shape

(400, 400, 3)

In [20]:
# 加载数据

In [21]:
def batch_obtain(dir_name):
    # dir_name = "data-set/data-moveImg-400x400"
    import re
    data_keep = []

    ink_code = open( './'+dir_name+'/ink-code.txt' )
    for lines in ink_code:
        # print lines
        searchObj = re.search( r'create\_picture\((\d+)\, (\d+)\, \"moveImg\-(\d+)\.bmp\"\)\;', lines, re.M|re.I)
        if searchObj: 
            tmp=[]
            for i in range(3):
                tmp.append( int( searchObj.group(i+1) ) )
            data_keep.append( tmp )

    data_keep=np.array( data_keep )

    x_batch = []
    y_batch = []

    for i in range( data_keep.shape[0] ): # data_keep.shape[0]
        image = cv2.imread( './'+dir_name+'/moveImg-'+str( data_keep[i][2] )+'.bmp' )
        x_batch.append( image )
        y_batch.append( [data_keep[i][0], data_keep[i][1]] )

    y_batch = np.array( y_batch )

    return x_batch, y_batch

In [22]:
x_batch, y_batch = batch_obtain( "data-set/data-moveImg-400x400" )

In [23]:
y_batch.shape

(100, 2)

In [24]:
# 开始训练

In [25]:
init = tf.global_variables_initializer() 

In [26]:
sess = tf.Session()

In [27]:
init.run( session=sess )

In [28]:
out_save = []

In [33]:
for _ in range(10):
    for _ in range(100):
        sess.run([ train ], { x: [im], y_input: [ [197, 165] ]})
    out_save.append( sess.run( [f2],{ x:[im] } ) )

In [34]:
out_save

[[array([[ 3.08174968,  4.88120508]], dtype=float32)],
 [array([[  9.81607151,  13.49610329]], dtype=float32)],
 [array([[ 23.35027885,  28.58366203]], dtype=float32)],
 [array([[ 45.71007538,  50.43025589]], dtype=float32)],
 [array([[ 71.61516571,  75.91602325]], dtype=float32)],
 [array([[  98.54956818,  101.8483429 ]], dtype=float32)],
 [array([[ 124.02584839,  125.26886749]], dtype=float32)],
 [array([[ 145.94062805,  143.79598999]], dtype=float32)],
 [array([[ 163.11135864,  156.35891724]], dtype=float32)],
 [array([[ 175.43399048,  163.41975403]], dtype=float32)],
 [array([[ 183.67446899,  166.51167297]], dtype=float32)],
 [array([[ 188.92521667,  167.32116699]], dtype=float32)],
 [array([[ 192.17753601,  167.09855652]], dtype=float32)],
 [array([[ 194.16114807,  166.56134033]], dtype=float32)],
 [array([[ 195.35813904,  166.03813171]], dtype=float32)],
 [array([[ 196.07174683,  165.63778687]], dtype=float32)],
 [array([[ 196.48968506,  165.36801147]], dtype=float32)],
 [array([